In [ ]:
!pip install pandas scikit-learn

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

# Función para preparar los datos diarios
def preparar_datos(fecha_inicio):
    # Crear un rango de fechas de seis meses desde la fecha de inicio
    fecha_inicio = pd.to_datetime(fecha_inicio)
    fechas = pd.date_range(start=fecha_inicio, periods=6 * 30, freq='D')
    
    # Crear un DataFrame con las fechas y datos simulados
    datos = pd.DataFrame({'datetime': fechas})
    datos['year'] = datos['datetime'].dt.year
    datos['month'] = datos['datetime'].dt.month
    datos['day'] = datos['datetime'].dt.day

    # Simular datos de temperatura, humedad y precipitación
    datos['temp'] = 20 + 5 * np.sin(datos.index * 0.1)
    datos['humidity'] = 50 + 10 * np.cos(datos.index * 0.1)
    datos['precip'] = np.random.rand(len(datos))

    return datos

# Función para entrenar y devolver los modelos y predicciones
def entrenar_predecir(fecha_inicio):
    datos_intervalo = preparar_datos(fecha_inicio)

    # Seleccionar las características relevantes
    X = datos_intervalo[['temp', 'humidity', 'precip']].fillna(0)

    # Variables objetivo
    y_humedad = datos_intervalo['humidity']
    y_precip = datos_intervalo['precip']
    y_drought = (datos_intervalo['precip'] < 1).astype(int)

    # Dividir el conjunto de datos
    X_train, X_test, y_train_humedad, _ = train_test_split(X, y_humedad, test_size=0.2, random_state=42)
    X_train_precip, X_test_precip, y_train_precip, _ = train_test_split(X, y_precip, test_size=0.2, random_state=42)
    X_train_drought, X_test_drought, y_train_drought, _ = train_test_split(X, y_drought, test_size=0.2, random_state=42)

    # Entrenar los modelos
    modelo_humidity = RandomForestRegressor(n_estimators=100, random_state=42)
    modelo_humidity.fit(X_train, y_train_humedad)

    modelo_precip = RandomForestRegressor(n_estimators=100, random_state=42)
    modelo_precip.fit(X_train_precip, y_train_precip)

    modelo_drought = RandomForestRegressor(n_estimators=100, random_state=42)
    modelo_drought.fit(X_train_drought, y_train_drought)

    # Obtener predicciones
    pred_humedad = modelo_humidity.predict(X)
    pred_precip = modelo_precip.predict(X)
    pred_drought = modelo_drought.predict(X)
    
    # Consolidar datos y predicciones en un solo DataFrame
    datos_predicciones = pd.DataFrame({
        'Fecha': datos_intervalo['datetime'],
        'temperature': datos_intervalo['temp'],    # Valor real
        'humidity': pred_humedad,                  # Predicción
        'rainfall': pred_precip,                   # Predicción
        'drought': pred_drought                    # Predicción
    })
    
    return datos_predicciones

# Ejemplo de ejecución para verificar el modelo de clima
fecha_prueba = '2024-11-01'
datos_predicciones = entrenar_predecir(fecha_prueba)
print(datos_predicciones.head())  # Mostrar las primeras filas para ver el formato

In [ ]:
# Ejemplo para probar el envío de datos con una fecha específica
fecha_prueba = '2024-11-01'  # Cambia esta fecha para probar con otras
datos_intervalo, pred_humedad, pred_precip, pred_drought = entrenar_predecir(fecha_prueba)

# Mostrar las primeras filas de la tabla de resultados para verificar
crear_tabla_resultados(datos_intervalo, pred_humedad, pred_precip, pred_drought)


In [ ]:
import joblib

# Función para guardar los modelos entrenados
def guardar_modelos(modelo_humidity, modelo_precip, modelo_drought):
    # Guardar los modelos en archivos .pkl
    joblib.dump(modelo_humidity, 'modelo_humidity_V2.pkl')
    joblib.dump(modelo_precip, 'modelo_precip.pkl_V2')
    joblib.dump(modelo_drought, 'modelo_drought_V2.pkl')
    print("Modelos guardados exitosamente.")

# Entrenar los modelos y guardar
fecha_prueba = '2024-11-01'
datos_intervalo, pred_humedad, pred_precip, pred_drought = entrenar_predecir(fecha_prueba)

# Llama a la función para guardar los modelos
guardar_modelos(modelo_humidity, modelo_precip, modelo_drought)
